# 🎯 Mini RAG: Persistindo e Reutilizando Índices Vetoriais

## O que vamos aprender?

Neste notebook, você vai aprender a **salvar e carregar** índices vetoriais do FAISS, um passo essencial para construir aplicações **RAG (Retrieval Augmented Generation)** em produção.

### O que é RAG?

**RAG = Retrieval Augmented Generation**

É uma técnica que combina:
1. 🔍 **Busca vetorial** (o que fizemos nos labs anteriores)
2. 🤖 **LLM** (ChatGPT, Gemini, etc.) para gerar respostas

**Fluxo RAG completo:**

```text
Pergunta do usuário
    ↓
Busca vetorial (FAISS) → Encontra documentos relevantes
    ↓
Contexto + Pergunta → Enviado para LLM
    ↓
LLM gera resposta baseada no contexto
    ↓
Resposta final ao usuário
```

### Por que persistir índices?

**Problema sem persistência:**
```python
# Toda vez que você reinicia o programa:
vector_store = FAISS.from_texts(meus_textos, embeddings)
# ↑ Precisa recalcular TODOS os embeddings (caro e lento!)
```

**Solução com persistência:**
```python
# Uma vez:
vector_store.save_local("meu_indice")

# Depois, sempre que precisar:
vector_store = FAISS.load_local("meu_indice", embeddings)
# ↑ Instantâneo! Não recalcula nada
```

### Benefícios

✅ **Economia:** Não paga API repetidamente  
✅ **Velocidade:** Carrega em milissegundos  
✅ **Escalabilidade:** Índices podem ter milhões de documentos  
✅ **Produção:** Essencial para apps reais

### O que faremos

1. Criar um índice vetorial (como antes)
2. **Salvar no disco** (novidade!)
3. **Carregar do disco** (novidade!)
4. Usar normalmente para buscas

💡 **Analogia:** É como salvar um jogo - você não quer recomeçar do zero toda vez!

In [1]:
import os
import requests
from pathlib import Path
from dotenv import load_dotenv
from langchain_community.vectorstores import FAISS

from langchain_google_genai import GoogleGenerativeAIEmbeddings

e:\01-projetos\11-work\11.34-engenharia-vetorial\.venv\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.5) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


## 📦 Imports e Configuração

Bibliotecas necessárias:
- `os`, `Path`, `load_dotenv`: Gerenciamento de arquivos e variáveis de ambiente
- `FAISS`: Banco de dados vetorial (com suporte a salvar/carregar!)
- `GoogleGenerativeAIEmbeddings`: API do Google para embeddings (gratuita!)

### Por que Google Generative AI?

Neste lab usamos Google em vez de OpenAI porque:
- ✅ **Tier gratuito generoso** (1500 requests/dia)
- ✅ **Qualidade excelente** (768 dimensões)
- ✅ **Sem cartão de crédito** para começar

**Como obter API key:**
1. Acesse [aistudio.google.com/app/apikey](https://aistudio.google.com/app/apikey)
2. Clique "Create API Key"
3. Adicione no `.env`: `GOOGLE_API_KEY=AIza...`

In [2]:
# 2) Configuração e carregamento do .env (simplificado)
env_path = Path.cwd().joinpath('..', '..', '.env').resolve()
if env_path.exists():
    load_dotenv(env_path)
    print(f'🔎 .env carregado -> {env_path.resolve()}')
else:
    print('⚠️  .env não encontrado. Defina as variáveis de ambiente manualmente.')


🔎 .env carregado -> E:\01-projetos\11-work\11.34-engenharia-vetorial\.env


## 🔐 Carregando Variáveis de Ambiente

Carrega a `GOOGLE_API_KEY` do arquivo `.env`.

**Estrutura do `.env`:**
```bash
GOOGLE_API_KEY=AIzaSy...
```

Se você vir ⚠️, crie o arquivo `.env` na raiz do projeto com sua API key do Google.

In [3]:

meus_textos = [
    "O novo iPhone 15 tem uma lente periscópica incrível.",    # Tecnologia
    "Para fazer um bolo macio, bata as claras em neve.",       # Culinária
    "O atacante chutou a bola no ângulo e foi gol.",           # Esporte
    "A placa de vídeo RTX 4090 roda jogos em 4K.",             # Tecnologia
    "Receita de lasanha à bolonhesa com muito queijo."         # Culinária
]

## 📄 Passo 1: Preparando os Documentos

Nosso dataset de teste com 5 documentos de categorias diferentes.

**Importante:** Em uma aplicação RAG real, esses textos viriam de:
- 📑 PDFs de documentação
- 🌐 Base de conhecimento da empresa
- 💬 FAQs e tickets de suporte
- 📊 Manuais técnicos

💡 **Conceito RAG:** Quanto mais documentos relevantes você indexar, melhor o LLM conseguirá responder perguntas específicas do seu domínio!

In [4]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")


## 🧠 Passo 2: Inicializando o Modelo de Embeddings

Criamos uma instância do modelo de embeddings do Google.

**Modelo:** `text-embedding-004`
- **Dimensões:** 768 (menor que OpenAI, mas ainda excelente)
- **Qualidade:** ⭐⭐⭐⭐⭐ Estado da arte
- **Custo:** Gratuito até 1500 requests/dia
- **Velocidade:** ~100-200ms por request

### Como funciona?

```
Texto → 🌐 API Google → Vetor [768 números]
```

Cada texto será transformado em um vetor de 768 dimensões que captura seu significado semântico.

In [5]:
vector_store = FAISS.from_texts(meus_textos, embeddings)

## 🗄️ Passo 3: Criando o Índice Vetorial

Transformamos os textos em vetores e criamos o índice FAISS **em memória**.

### O que acontece internamente?

```text
Para cada texto em meus_textos:
  1. Envia para API do Google
  2. Recebe vetor de 768 dimensões
  3. FAISS armazena o vetor em RAM

Resultado:
  vector_store = Índice FAISS com 5 vetores em memória
```

**Importante:** Neste ponto, o índice existe **apenas na RAM**! Se você fechar o programa, perde tudo.

⏱️ **Tempo:** ~1-2 segundos (5 chamadas à API do Google)

In [11]:
FAISS_PATH = Path.cwd().joinpath('..', '..', 'data', 'meu_indice_faiss')

vector_store.save_local(str(FAISS_PATH))
print("Banco vetorial salvo com sucesso!")

Banco vetorial salvo com sucesso!


## 💾 Passo 4: Salvando o Índice no Disco (PERSISTÊNCIA!)

Esta é a **parte mais importante** para aplicações RAG em produção!

### O que acontece aqui?

```python
vector_store.save_local(str(FAISS_PATH))
```

**Internamente:**
1. FAISS cria uma pasta: `data/meu_indice_faiss/`
2. Salva 2 arquivos:
   - `index.faiss` → Os vetores e estrutura de índice
   - `index.pkl` → Metadados (textos originais, etc.)

### Estrutura de arquivos criada:

```text
data/
└── meu_indice_faiss/
    ├── index.faiss  ← ~50KB (5 vetores × 768 dims × 4 bytes)
    └── index.pkl    ← ~2KB (textos originais)
```

### Por que isso é importante?

**Sem persistência:**
```python
# Toda execução:
vector_store = FAISS.from_texts(textos, embeddings)
# ↑ 5 chamadas à API + 1-2 segundos
```

**Com persistência:**
```python
# Apenas uma vez:
vector_store.save_local("path")

# Depois, sempre:
vector_store = FAISS.load_local("path", embeddings)
# ↑ 0 chamadas à API + ~10ms
```

### Benefícios em produção

| Métrica | Sem Persistência | Com Persistência |
|---------|------------------|------------------|
| **Tempo de startup** | Minutos (muitos docs) | Milissegundos |
| **Custo de API** | Toda vez | Uma vez (na criação) |
| **Escalabilidade** | Limitada | Milhões de docs |
| **Confiabilidade** | Dependente de API | Funciona offline |

💡 **Analogia:** É como salvar um documento do Word - você não quer redigitar tudo cada vez que abrir!

In [13]:
# CARREGAR do disco
# O parâmetro 'allow_dangerous_deserialization' é necessário em versões recentes
# para confirmar que você confia no arquivo que está carregando.
novo_db = FAISS.load_local(
    str(FAISS_PATH), 
    embeddings, 
    allow_dangerous_deserialization=True
)

## 📂 Passo 5: Carregando o Índice do Disco

Agora vamos **carregar** o índice que salvamos anteriormente!

### O que acontece aqui?

```python
novo_db = FAISS.load_local(str(FAISS_PATH), embeddings, allow_dangerous_deserialization=True)
```

**Internamente:**
1. FAISS lê os arquivos da pasta `data/meu_indice_faiss/`
2. Reconstrói o índice em memória (~10ms)
3. Pronto para uso instantâneo!

### Parâmetros importantes

**`embeddings`**: Necessário para futuras queries
- Não recalcula nada dos documentos existentes
- Só será usado quando você buscar algo novo

**`allow_dangerous_deserialization=True`**: Confirmação de segurança
- FAISS usa pickle para serializar
- Pickle pode executar código malicioso se vier de fonte não confiável
- Como você mesmo criou o índice, é seguro ✅

### Fluxo completo de uma aplicação RAG

```text
Primeira execução (setup):
  1. Carregar documentos
  2. Criar embeddings (via API)
  3. Criar índice FAISS
  4. Salvar no disco ← Você está aqui!

Execuções subsequentes (produção):
  5. Carregar índice do disco ← Estamos fazendo isso agora!
  6. Receber query do usuário
  7. Buscar documentos relevantes
  8. Enviar contexto + query para LLM
  9. Retornar resposta
```

### Performance

| Operação | Tempo (5 docs) | Tempo (10k docs) | Tempo (1M docs) |
|----------|----------------|------------------|-----------------|
| **Criar índice** | ~2s | ~30s | ~30min |
| **Salvar** | ~50ms | ~200ms | ~5s |
| **Carregar** | ~10ms | ~100ms | ~2s |
| **Buscar** | <1ms | ~5ms | ~50ms |

💡 **Observe:** Carregar é **muito mais rápido** que criar!

In [16]:
# 3. Usar normalmente
consulta = "Sugestão de celular"
resultados = novo_db.similarity_search(consulta, k=2)

## 🔍 Passo 6: Usando o Índice Carregado

Agora vamos fazer uma busca usando o índice que **carregamos do disco**!

### A Query

```python
consulta = "Sugestão de celular"
```

**Desafio semântico:**
- A palavra "iPhone" não aparece na query
- A palavra "celular" sim
- O modelo precisa entender que iPhone é um celular

### O que acontece internamente?

1. **Query vira vetor (via API Google):**
   ```text
   "Sugestão de celular" → 🌐 API call → [0.15, -0.32, ..., 0.47]
   ```

2. **FAISS busca (local, sem API):**
   ```text
   Compara o vetor da query com os 5 vetores armazenados
   Calcula distâncias:
     - iPhone: 0.3 ← Mais próximo!
     - RTX 4090: 1.2
     - Bolo: 2.5
     - Gol: 2.8
     - Lasanha: 2.6
   ```

3. **Retorna top-k (k=2):**
   ```
   [iPhone, RTX 4090]
   ```

### Parâmetro k

- `k=1`: Retorna apenas o mais similar
- `k=2`: Retorna os 2 mais similares
- `k=5`: Retorna todos (nosso dataset tem 5)

💡 **Dica:** Em RAG, geralmente usa-se `k=3` ou `k=5` para dar contexto suficiente ao LLM sem sobrecarregar.

In [17]:
print(resultados[0].page_content)

O novo iPhone 15 tem uma lente periscópica incrível.


## 📊 Passo 7: Visualizando o Resultado

Vamos ver o documento mais relevante (índice 0 = primeiro resultado).

**Esperado:** Deve retornar algo sobre iPhone, pois é o documento sobre celular!

### Interpretando o resultado

Se você ver:
- ✅ `"O novo iPhone 15..."` → Perfeito! A busca funcionou
- ❌ Qualquer outro texto → Algo deu errado (improvável com Google)

### Próximo passo: RAG completo

Em um RAG de verdade, você faria:

```python
# 1. Buscar documentos relevantes (já fizemos!)
resultados = novo_db.similarity_search(consulta, k=3)

# 2. Montar o contexto
contexto = "\n".join([doc.page_content for doc in resultados])

# 3. Criar prompt para LLM
prompt = f"""
Baseado no seguinte contexto:
{contexto}

Responda a pergunta: {consulta}
"""

# 4. Enviar para LLM (Gemini, GPT, etc.)
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-pro")
resposta = llm.invoke(prompt)

print(resposta.content)
# → "Eu recomendo o iPhone 15, que possui uma lente periscópica incrível..."
```

💡 **Esse é o poder do RAG:** O LLM responde com base nos **seus** documentos, não apenas no conhecimento geral!

## 🎓 Resumo e Conceitos-Chave

### O que aprendemos

✅ **Persistência de índices** - Salvar e carregar FAISS do disco  
✅ **Economia de API** - Não recalcular embeddings toda vez  
✅ **Performance** - Carregar é 100x mais rápido que criar  
✅ **Fundamentos de RAG** - Base para aplicações com LLM

### Fluxo completo que implementamos

```text
1. Criar embeddings (via API Google)
   ↓
2. Criar índice FAISS (em RAM)
   ↓
3. Salvar no disco (.save_local) ← PERSISTÊNCIA!
   ↓
4. Carregar do disco (.load_local) ← REUTILIZAÇÃO!
   ↓
5. Fazer buscas normalmente
```

### Diferenças: Labs anteriores vs. Este

| Aspecto | Labs 1.4-1.6 | Este Lab (1.7) |
|---------|--------------|----------------|
| **Índice** | Só em RAM | Salvo em disco |
| **Reiniciar programa** | Perde tudo | Mantém tudo |
| **Startup** | Lento (recria) | Rápido (carrega) |
| **Produção** | ❌ Inviável | ✅ Pronto |

### Aplicações reais de RAG

🤖 **Chatbots corporativos**
- Indexa documentação interna
- LLM responde baseado nos docs da empresa

📚 **Assistentes de estudo**
- Indexa livros, apostilas, anotações
- LLM explica conceitos baseado no material

🔧 **Suporte técnico**
- Indexa manuais, FAQs, tickets antigos
- LLM sugere soluções baseadas em casos similares

🏥 **Assistência médica**
- Indexa prontuários, estudos científicos
- LLM auxilia diagnósticos (com supervisão humana!)



### 🧪 Experimentos para tentar

#### Experimento 1: Adicionar mais documentos
```python
novos_textos = [
    "Samsung Galaxy S23 tem câmera de 200MP",
    "Xiaomi Redmi Note 12 é um bom custo-benefício",
]

# Criar novo índice com todos os documentos
todos_textos = meus_textos + novos_textos
vector_store = FAISS.from_texts(todos_textos, embeddings)
vector_store.save_local(str(FAISS_PATH))

# Agora tem 7 documentos!
```



#### Experimento 2: Atualizar índice existente
```python
# Carregar índice existente
db = FAISS.load_local(str(FAISS_PATH), embeddings, allow_dangerous_deserialization=True)

# Adicionar novos documentos
novos_docs = ["Novo documento..."]
db.add_texts(novos_docs)

# Salvar novamente (agora com os novos docs)
db.save_local(str(FAISS_PATH))
```



#### Experimento 3: Múltiplos índices
```python
# Índice para tecnologia
tech_db = FAISS.from_texts(textos_tech, embeddings)
tech_db.save_local("indices/tecnologia")

# Índice para culinária
food_db = FAISS.from_texts(textos_culinaria, embeddings)
food_db.save_local("indices/culinaria")

# Carregar conforme necessário
tech = FAISS.load_local("indices/tecnologia", embeddings, allow_dangerous_deserialization=True)
```



#### Experimento 4: RAG completo (desafio!)
```python
# Combine tudo que aprendeu:
# 1. Carregar índice
# 2. Fazer busca
# 3. Enviar contexto para LLM (Gemini)
# 4. Retornar resposta gerada

from langchain_google_genai import ChatGoogleGenerativeAI

def rag_query(pergunta, k=3):
    # Buscar documentos relevantes
    db = FAISS.load_local(str(FAISS_PATH), embeddings, allow_dangerous_deserialization=True)
    docs = db.similarity_search(pergunta, k=k)
    
    # Montar contexto
    contexto = "\n".join([d.page_content for d in docs])
    
    # Prompt para LLM
    prompt = f"Contexto:\n{contexto}\n\nPergunta: {pergunta}\nResposta:"
    
    # Gerar resposta
    llm = ChatGoogleGenerativeAI(model="gemini-pro")
    resposta = llm.invoke(prompt)
    
    return resposta.content

# Testar
print(rag_query("Qual é o melhor celular?"))
```



### 💡 Boas práticas em produção

1. **Versionamento de índices**
   ```python
   vector_store.save_local("indices/v1.0.0")
   vector_store.save_local("indices/v1.1.0")  # Após adicionar docs
   ```

2. **Backup automático**
   ```python
   import shutil
   from datetime import datetime
   
   backup_path = f"backups/index_{datetime.now().strftime('%Y%m%d')}"
   shutil.copytree(FAISS_PATH, backup_path)
   ```

3. **Metadados**
   ```python
   # Salvar informações sobre o índice
   metadata = {
       "created_at": datetime.now(),
       "num_docs": len(meus_textos),
       "model": "text-embedding-004",
       "dimensions": 768
   }
   
   import json
   with open(FAISS_PATH / "metadata.json", "w") as f:
       json.dump(metadata, f)
   ```

4. **Monitoramento**
   ```python
   import os
   
   # Verificar tamanho do índice
   index_size = os.path.getsize(FAISS_PATH / "index.faiss")
   print(f"Tamanho do índice: {index_size / 1024:.2f} KB")
   ```

💡 **Dica final:** O RAG é a técnica mais popular para fazer LLMs responderem com informações específicas do seu negócio. Domine isso e você terá um skill muito valorizado no mercado!